In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

# Simulate Borwnian Motion and square root interpolate the resulting path
# Define a function to create a Gaussian Orthoganal Ensemble
def Gaussian_Orthogonal_Ensemble(n, **kwargs):
    
    '''
    Generates a nxn Gaussian Orthogonal Ensemble
    '''
    
    # Define a normally distributed array
    H = np.random.normal(0, 1, size=(n,n))
    
    # Make it symmetric
    H = np.triu(H) + np.triu(H, k=1).T
    
    # Multiply the diagonal values by a factor of sqrt(2)
    np.fill_diagonal(H, np.sqrt(2) * np.diag(H))
    
    return H

def eigenvalues(n, **kwargs):
    
    '''
    Computes the eigenvalues for a nxn Gaussian Orthogonal Ensemble
    '''
    
    eigs = np.linalg.eig(Gaussian_Orthogonal_Ensemble(n))[0]
    
    return eigs

def sqrt_interp(t, p1, p2, **kwargs):
    
    # Create a copy of each point
    pt1 = p1.copy()
    pt2 = p2.copy()
    
    # Project point 1 to x axis
    pt1[1] = 0
    pt2[1] = p2[1] - p1[1]
    
    # Calculate positive and reflect over y axis if needed
    if pt2[1] - pt1[1] < 0:
        
        sqrt_interp = -1 * np.sqrt(np.interp(t, [pt1[0], pt2[0]], [pt1[1]**2, pt2[1]**2]))
    
    else:
        
        sqrt_interp = np.sqrt(np.interp(t, [pt1[0], pt2[0]], [pt1[1]**2, pt2[1]**2]))
    
    # Undo projection
    sqrt_interp += p1[1]
    
    # Reflect and translate
    sqrt_interp = -1 * sqrt_interp + p1[1] + p2[1]
    
    return sqrt_interp

def n_Brownian_Motions_GOE(n, n_i, T, steps, dt):
    
    '''
    Simulates the Brownian Motion of n particles with starting positions of eigenvalues of a GOE.
    '''
    
    # Create an array of positions +  assign initial values
    positions = np.zeros((int(np.ceil((T+dt) / dt)), n))
    positions[0] = n_i
    
    # For each timestep after the initial
    for i in range(len(positions) - 1):
        
        # Generate some eigenvalues for the motion
        eigs = np.sort(eigenvalues(n))
        
        # Calculate the Coulombic interation
        for j in range(n):
            
            coulombic = 0
            
            for k in range(n):
            
                if k != j:
                
                    coulombic += 1 / (positions[i, j] - positions[i, k])
            
            # Assign that interaction to each particle and calculate the path for that timestep
            positions[i+1, j] = positions[i, j] + np.sqrt(2 * dt / n) * eigs[j] + coulombic / n
    
    # Slice out the wanted values
    positions_interp = positions[::int(T / dt) // steps]
        
    return positions, positions_interp

############################
# Brownian Motion Conditions
############################
n = 1
n_i = np.sort(eigenvalues(n))
T = 1
steps = 10 # Stepsize for sqrt interpolation
dt = 0.01
interp_mesh_size = 1000

###############
# Interpolating
###############
positions, positions_interp = n_Brownian_Motions_GOE(n, n_i, T, steps, dt)

interp_brownian = np.array([])
t_brownian = np.array([])

for j in range((int(T / dt) // steps)):
        
        # Calculate where each 'point' is [t, y], same as in the example above.
        point1 = [T - (j + 1) * dt * steps, positions[-1 * (j+1) * steps-1, 0]]
        point2 = [T - j * dt * steps, positions[-1 * j * steps-1, 0]]

        t_brownian = np.append(t_brownian, np.linspace(point1[0], point2[0], interp_mesh_size))
        interp_brownian = np.append(interp_brownian, sqrt_interp(np.linspace(point1[0], point2[0], interp_mesh_size), point1, point2))
        
t_brownian = np.flip(t_brownian)

# Lambda definition
def lambda_(t):
    
    return np.interp(t, t_brownian, interp_brownian)

# Derivative definition
def complex_function(t,y):
    
    x = y[0]
    y = y[1]

    dxdt = 0.5 * ( -2 * (x - lambda_(t)) / ( (x - lambda_(t)) ** 2 + y ** 2 ))
    
    dydt = 0.5 * ( 2 * y / ( (x - lambda_(t)) ** 2 + y ** 2 ))

    return np.array([dxdt, dydt])

x_points = np.concatenate((np.flip(-1 * np.logspace(-2, np.log10(2), 30)), np.logspace(-2, np.log10(2), 30)))
y_points = np.logspace(-8, np.log10(2), 40)

singularity = np.array([1e-10, 1e-10])
t_max = 1
anim_times = np.linspace(0,t_max, 101)

sol_3D = np.zeros((len(anim_times), len(x_points), len(y_points), 2))

for i in range(len(x_points)):
    
    for j in range(len(y_points)):
        
        sol_3D[0,i,j] = np.array([x_points[i], y_points[j]])
        
print('Simulating Grid')
for k in range(len(anim_times) - 1):
    
    print(f'{k+1}/{len(anim_times) - 1}')

    for i in range(len(x_points)):
    
        for j in range(len(y_points)):
            
            t_span = [k * t_max / len(anim_times), (k + 1) * t_max / len(anim_times)]
            
            z0 = np.array([sol_3D[k,i,j,0],sol_3D[k,i,j,1]])
            
            sol = solve_ivp(complex_function, t_span, z0, method='BDF', atol=1e-15, rtol=1e-12)
            
            x = sol.y[0]
            y = sol.y[1]
        
            sol_3D[k+1,i,j,0] = x[-1]
            sol_3D[k+1,i,j,1] = y[-1]

# Track the singularity
singularity_sol = np.zeros((len(anim_times), 2))
singularity_sol[0] = singularity

print('Simulating Singularity')
for k in range(len(anim_times) - 1):
    
    print(f'{k+1}/{len(anim_times) - 1}')
            
    t_span = [k * t_max / len(anim_times), (k + 1) * t_max / len(anim_times)]
            
    z0 = np.array([singularity_sol[k,0],singularity_sol[k,1]])
            
    sol = solve_ivp(complex_function, t_span, z0, method='BDF', atol=1e-15, rtol=1e-12)
            
    x = sol.y[0]
    y = sol.y[1]
        
    singularity_sol[k+1,0] = x[-1]
    singularity_sol[k+1,1] = y[-1]

print('Animating')
# Create a function to generate your plots
def generate_plot(frame):
    
    print(f'{frame + 1} / {len(anim_times)-1}')

    # Clear the current axes
    plt.cla()
    
    # Plot the grid
    for i in range(len(x_points)):
    
        plt.plot(sol_3D[frame,i,:,0], sol_3D[frame,i,:,1], lw=0.4, color='tab:blue')
        
    for j in range(len(y_points)):
            
        plt.plot(sol_3D[frame,:,j,0], sol_3D[frame,:,j,1], lw=0.4, color='tab:blue')    

    # Plot the singularity path
    plt.plot(singularity_sol[:frame,0], singularity_sol[:frame,1], color='tab:red')
    plt.scatter(singularity_sol[frame,0], singularity_sol[frame,1], s=0.4, color='tab:red')
            
    # Set plot title and labels, if desired
    plt.title(f"t={anim_times[frame]:0.2f}")

    # Return the plot or figure object
    return plt

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10,10))

# Create the animation
anim = animation.FuncAnimation(fig, generate_plot, frames=len(anim_times), interval=200)

# Set up the writer
writer = PillowWriter(fps=10)

# Save the animation as an gif file
anim.save('animation_grid_one_brownian_driver.gif', writer=writer)
plt.close()



Simulating Grid
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100
Simulating Singularity
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/10